<img src="http://imgur.com/1ZcRyrc.png" style="float: left; margin: 20px; height: 55px">

# Project 2 - Singapore Housing Data and Kaggle Challenge

## Part 3 - Modelling and Kaggle Submission

### Contents:
- [1.1 Data Import](#1.1-Data-Import)
- [1.2 Preprocessing Test Data](#1.2-Preprocessing-Test-Data)
- [2 Kaggle Submission](#2-Kaggle-Submission)
- [3 Bonus: Creation of a Resale Price Predictor App](#3-Bonus:-Creation-of-a-Resale-Price-Predictor-App)

## 1. Modelling

### 1.1 Data Import

In [35]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import math
import plotly.express as px

from sklearn.linear_model import LinearRegression
from sklearn.linear_model import RidgeCV, LassoCV
from sklearn import metrics
from sklearn.metrics import r2_score
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import make_column_transformer
from sklearn.model_selection import train_test_split, cross_val_score, cross_val_predict
from joblib import dump
import joblib

import warnings
warnings.filterwarnings('ignore')

import scipy.stats as stats
import statsmodels.api as sm

import pickle

In [36]:
# Importing data.
df_test = pd.read_csv('../datasets/test_filtered_1.csv', index_col=0)
df = df_test.iloc[:, 2:]

### 1.2 Preprocessing Test Data

In [37]:
# Set max colums display to see all the columns
pd.options.display.max_columns = 28

In [38]:
# Inspecting the dataframe
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 16737 entries, 0 to 16736
Data columns (total 27 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   floor_area_sqft            16737 non-null  float64
 1   planning_area              16737 non-null  object 
 2   city_distance              16737 non-null  float64
 3   lease_commence_date        16737 non-null  int64  
 4   tranc_year                 16737 non-null  int64  
 5   mid_storey                 16737 non-null  int64  
 6   max_floor_lvl              16737 non-null  int64  
 7   commercial                 16737 non-null  int64  
 8   market_hawker              16737 non-null  int64  
 9   multistorey_carpark        16737 non-null  int64  
 10  precinct_pavilion          16737 non-null  int64  
 11  total_dwelling_units       16737 non-null  int64  
 12  mall_nearest_distance      16737 non-null  float64
 13  mall_within_1km            16737 non-null  float64


In [39]:
df.head()

,floor_area_sqft,planning_area,city_distance,lease_commence_date,tranc_year,mid_storey,max_floor_lvl,commercial,market_hawker,multistorey_carpark,precinct_pavilion,total_dwelling_units,mall_nearest_distance,mall_within_1km,hawker_nearest_distance,hawker_within_1km,mrt_interchange,mrt_nearest_distance,bus_interchange,bus_stop_nearest_distance,pri_sch_nearest_distance,vacancy,pri_sch_affiliation,sec_sch_nearest_dist,cutoff_point,affiliation,flat_type
0,904.176,Yishun,17.277628,1987,2012,8,12,1,0,0,0,132,877.431572,2.0,687.576779,1.0,0,686.660434,0,75.683952,426.467910,92,0,156.322353,218,0,4 ROOM
1,1205.568,Jurong West,18.311735,2008,2019,5,14,0,0,0,0,53,534.037705,1.0,2122.346226,0.0,0,169.478175,0,88.993058,439.756851,45,0,739.371688,199,0,5 ROOM
2,731.952,Ang Mo Kio,10.130258,1980,2013,8,12,0,0,0,0,218,817.050453,2.0,152.287621,3.0,0,694.220448,1,86.303575,355.882207,36,0,305.071191,245,0,3 ROOM
3,1044.108,Woodlands,19.784959,1979,2017,2,14,0,0,0,0,104,1272.737194,0.0,501.892158,1.0,0,1117.203587,0,108.459039,929.744711,54,0,433.454591,188,0,4 ROOM
4,1108.692,Bukit Batok,14.239005,1985,2016,17,25,0,0,0,0,144,1070.963675,0.0,437.593564,2.0,0,987.976010,0,113.645431,309.926934,40,0,217.295361,223,0,4 ROOM


In [40]:
# Create a feature 'age_sold' which is the age when the flat was sold which is different from the age of the flat to the year 2021
df['age_sold'] = df['tranc_year'] - df['lease_commence_date']
df

,floor_area_sqft,planning_area,city_distance,lease_commence_date,tranc_year,mid_storey,max_floor_lvl,commercial,market_hawker,multistorey_carpark,precinct_pavilion,total_dwelling_units,mall_nearest_distance,mall_within_1km,hawker_nearest_distance,hawker_within_1km,mrt_interchange,mrt_nearest_distance,bus_interchange,bus_stop_nearest_distance,pri_sch_nearest_distance,vacancy,pri_sch_affiliation,sec_sch_nearest_dist,cutoff_point,affiliation,flat_type,age_sold
0,904.176,Yishun,17.277628,1987,2012,8,12,1,0,0,0,132,877.431572,2.0,687.576779,1.0,0,686.660434,0,75.683952,426.467910,92,0,156.322353,218,0,4 ROOM,25
1,1205.568,Jurong West,18.311735,2008,2019,5,14,0,0,0,0,53,534.037705,1.0,2122.346226,0.0,0,169.478175,0,88.993058,439.756851,45,0,739.371688,199,0,5 ROOM,11
2,731.952,Ang Mo Kio,10.130258,1980,2013,8,12,0,0,0,0,218,817.050453,2.0,152.287621,3.0,0,694.220448,1,86.303575,355.882207,36,0,305.071191,245,0,3 ROOM,33
3,1044.108,Woodlands,19.784959,1979,2017,2,14,0,0,0,0,104,1272.737194,0.0,501.892158,1.0,0,1117.203587,0,108.459039,929.744711,54,0,433.454591,188,0,4 ROOM,38
4,1108.692,Bukit Batok,14.239005,1985,2016,17,25,0,0,0,0,144,1070.963675,0.0,437.593564,2.0,0,987.976010,0,113.645431,309.926934,40,0,217.295361,223,0,4 ROOM,31
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16732,882.648,Ang Mo Kio,10.803266,1980,2016,5,11,0,0,0,0,258,1120.535819,0.0,226.144646,2.0,0,674.098299,1,76.352127,556.368531,63,0,320.543569,188,0,3 ROOM,36
16733,990.288,Punggol,14.906527,2012,2017,17,16,0,0,0,0,80,519.046576,2.0,4254.303044,0.0,1,452.475847,1,81.731727,149.376171,60,0,423.996711,194,0,4 ROOM,5
16734,1184.040,Choa Chu Kang,17.673096,2002,2018,5,16,0,0,0,0,120,685.922630,2.0,3917.745229,0.0,0,739.510043,0,101.163779,208.840038,54,0,560.399339,212,0,5 ROOM,16
16735,904.176,Bishan,7.159508,1988,2020,8,9,0,0,0,0,104,846.849200,1.0,773.521588,1.0,1,877.694888,1,98.358542,300.015596,45,1,324.050692,232,0,4 ROOM,32


In [41]:
# Dropping columns 'precinct_pavilion', 'bus_interchange', 'affiliation'
df.drop(columns=['precinct_pavilion', 'bus_interchange', 'affiliation'], inplace=True)
df.head()

,floor_area_sqft,planning_area,city_distance,lease_commence_date,tranc_year,mid_storey,max_floor_lvl,commercial,market_hawker,multistorey_carpark,total_dwelling_units,mall_nearest_distance,mall_within_1km,hawker_nearest_distance,hawker_within_1km,mrt_interchange,mrt_nearest_distance,bus_stop_nearest_distance,pri_sch_nearest_distance,vacancy,pri_sch_affiliation,sec_sch_nearest_dist,cutoff_point,flat_type,age_sold
0,904.176,Yishun,17.277628,1987,2012,8,12,1,0,0,132,877.431572,2.0,687.576779,1.0,0,686.660434,75.683952,426.467910,92,0,156.322353,218,4 ROOM,25
1,1205.568,Jurong West,18.311735,2008,2019,5,14,0,0,0,53,534.037705,1.0,2122.346226,0.0,0,169.478175,88.993058,439.756851,45,0,739.371688,199,5 ROOM,11
2,731.952,Ang Mo Kio,10.130258,1980,2013,8,12,0,0,0,218,817.050453,2.0,152.287621,3.0,0,694.220448,86.303575,355.882207,36,0,305.071191,245,3 ROOM,33
3,1044.108,Woodlands,19.784959,1979,2017,2,14,0,0,0,104,1272.737194,0.0,501.892158,1.0,0,1117.203587,108.459039,929.744711,54,0,433.454591,188,4 ROOM,38
4,1108.692,Bukit Batok,14.239005,1985,2016,17,25,0,0,0,144,1070.963675,0.0,437.593564,2.0,0,987.976010,113.645431,309.926934,40,0,217.295361,223,4 ROOM,31


In [42]:
# Create interaction features
df['citydist*mallneardist'] = df['city_distance'] * df['mall_nearest_distance']
df['midstorey*floorarea'] = df['mid_storey'] * df['floor_area_sqft']
df.head()

,floor_area_sqft,planning_area,city_distance,lease_commence_date,tranc_year,mid_storey,max_floor_lvl,commercial,market_hawker,multistorey_carpark,total_dwelling_units,mall_nearest_distance,mall_within_1km,hawker_nearest_distance,hawker_within_1km,mrt_interchange,mrt_nearest_distance,bus_stop_nearest_distance,pri_sch_nearest_distance,vacancy,pri_sch_affiliation,sec_sch_nearest_dist,cutoff_point,flat_type,age_sold,citydist*mallneardist,midstorey*floorarea
0,904.176,Yishun,17.277628,1987,2012,8,12,1,0,0,132,877.431572,2.0,687.576779,1.0,0,686.660434,75.683952,426.467910,92,0,156.322353,218,4 ROOM,25,15159.936722,7233.408
1,1205.568,Jurong West,18.311735,2008,2019,5,14,0,0,0,53,534.037705,1.0,2122.346226,0.0,0,169.478175,88.993058,439.756851,45,0,739.371688,199,5 ROOM,11,9779.157110,6027.840
2,731.952,Ang Mo Kio,10.130258,1980,2013,8,12,0,0,0,218,817.050453,2.0,152.287621,3.0,0,694.220448,86.303575,355.882207,36,0,305.071191,245,3 ROOM,33,8276.931932,5855.616
3,1044.108,Woodlands,19.784959,1979,2017,2,14,0,0,0,104,1272.737194,0.0,501.892158,1.0,0,1117.203587,108.459039,929.744711,54,0,433.454591,188,4 ROOM,38,25181.053774,2088.216
4,1108.692,Bukit Batok,14.239005,1985,2016,17,25,0,0,0,144,1070.963675,0.0,437.593564,2.0,0,987.976010,113.645431,309.926934,40,0,217.295361,223,4 ROOM,31,15249.456860,18847.764


In [43]:
# Defining numerical columns and categorical columns to be passed through SS and OHE respectively
one_hot_encode_cols = ['planning_area', 'flat_type']
standard_scale_cols = ['floor_area_sqft', 'lease_commence_date', 'tranc_year', 'mid_storey',
                       'max_floor_lvl', 'total_dwelling_units', 'mall_nearest_distance', 
                       'mall_within_1km', 'hawker_nearest_distance', 'hawker_within_1km', 
                       'mrt_nearest_distance', 'bus_stop_nearest_distance', 
                       'pri_sch_nearest_distance', 'vacancy', 'sec_sch_nearest_dist', 
                       'cutoff_point', 'city_distance', 'age_sold', 'citydist*mallneardist', 
                       'midstorey*floorarea']
pass_through_cols = ['commercial', 'market_hawker', 'multistorey_carpark', 'mrt_interchange', 'pri_sch_affiliation']

In [44]:
# Instantiating OHE, SS and CT
ohe = OneHotEncoder(min_frequency = 1)
ss = StandardScaler()
ct = make_column_transformer(
    (ohe, one_hot_encode_cols),
    (ss, standard_scale_cols),
    remainder='passthrough'  # Do not transform the remaining columns
)
ct

ColumnTransformer(remainder='passthrough',
                  transformers=[('onehotencoder',
                                 OneHotEncoder(min_frequency=1),
                                 ['planning_area', 'flat_type']),
                                ('standardscaler', StandardScaler(),
                                 ['floor_area_sqft', 'lease_commence_date',
                                  'tranc_year', 'mid_storey', 'max_floor_lvl',
                                  'total_dwelling_units',
                                  'mall_nearest_distance', 'mall_within_1km',
                                  'hawker_nearest_distance',
                                  'hawker_within_1km', 'mrt_nearest_distance',
                                  'bus_stop_nearest_distance',
                                  'pri_sch_nearest_distance', 'vacancy',
                                  'sec_sch_nearest_dist', 'cutoff_point',
                                  'city_distance', 'age_sold',
                                  'citydist*mallneardist',
                                  'midstorey*floorarea'])])

In [45]:
# Fit and transform
X_transformed = ct.fit_transform(df)

In [46]:
# Load the Ridge regression model from the pickle file
ridge_model = joblib.load('ridge_model.pkl')

In [47]:
# Make predictions using the loaded model on the transformed data
predictions = ridge_model.predict(X_transformed)
predictions

array([[352577.61034896],
       [500019.498152  ],
       [348983.19431966],
       ...,
       [392755.43463254],
       [480947.69981722],
       [382478.09898854]])

## 2. Kaggle Submission

In [48]:
# Slicing 'id' column from df_test
test_id = df_test['id']
test_id

0        114982
1         95653
2         40303
3        109506
4        100149
          ...  
16732     23347
16733     54003
16734    128921
16735     69352
16736    146210
Name: id, Length: 16737, dtype: int64

In [49]:
# Creating a DataFrame from the test_id 
result = pd.DataFrame(test_id)
result

,id
0,114982
1,95653
2,40303
3,109506
4,100149
...,...
16732,23347
16733,54003
16734,128921
16735,69352


In [50]:
# Renaming the id column
result.rename(columns = {'id': 'Id'})

# Adding the prediction from test data alongside the id
result['Predicted'] = predictions
result

,id,Predicted
0,114982,352577.610349
1,95653,500019.498152
2,40303,348983.194320
3,109506,257691.449804
4,100149,488192.138521
...,...,...
16732,23347,357029.523160
16733,54003,500708.405527
16734,128921,392755.434633
16735,69352,480947.699817


In [51]:
# Final upload as csv
result.to_csv('../datasets/submission_ridge.csv', index = False)

### Kaggle Submission Screenshot

![Kaggle Submission Screenshot](../images/Kaggle_Submission_Screenshot.png)